In [4]:
from elasticsearch import Elasticsearch

client = Elasticsearch(
    "https://f0c40d3bde77471f83e6cace87537665.ap-southeast-1.aws.found.io",
    # api_key="Mk9odVlwY0JiN3JESEE4VzhkZ286WTNuZ29WR2pYMDF6em1DV193WVdkZw=="
    api_key="MnVpVVlwY0JiN3JESEE4V2FOaEk6X2NRRS1ESnM2V1A2TDJKQ25pb01NUQ=="
)

index_name = "cvedocs"

# # Delete if exists (optional cleanup step for testing)
# if client.indices.exists(index=index_name):
#     client.indices.delete(index=index_name)

# Create index with proper mapping
mappings = {
    "mappings": {
        "properties": {
            "text": {
                "properties": {
                    "cve_id": {"type": "keyword"},
                    "description": {"type": "text"},
                    "severity": {"type": "keyword"}
                }
            }
        }
    }
}

# mappings = {
#     "mappings": {
#         "properties": {
#             "text": {"type": "text"},  # not an object
#             "metadata": {
#                 "properties": {
#                     "cve_id": {"type": "keyword"},
#                     "severity": {"type": "keyword"},
#                     "cvss_score": {"type": "float"},
#                     "fix_version": {"type": "keyword"},
#                     "published_date": {"type": "date"},
#                     "last_updated": {"type": "date"},
#                     # Add more fields as needed
#                 }
#             }
#         }
#     }
# }

mappings = {
    "mappings": {
        "properties": {
              "text": { "type": "text" },
              "vector": {
                  "type": "dense_vector",
                  "dims": 768,
                  "index": True,
                  "similarity": "cosine"
  },
  "metadata": {
    "properties": {
      "id":{"type":"keyword"},
      "cve_id": { "type": "keyword" },
      "severity": { "type": "keyword" },
      "cvss_score": { "type": "float" },
      "fix_version": { "type": "keyword" },
      "published_date": { "type": "date" },
      "last_updated": { "type": "date" }
    }
  }
}}}
# client.indices.create(index=index_name, body=mappings)

# Sample document
# doc = {
#     "text": {
#         "cve_id": "CVE-2024-12345",
#         "description": "This is a test CVE vulnerability.",
#         "severity": "High"
#     }
# }

# Index the document
# response = client.index(index=index_name, document=doc)
# print("Document indexed:", response)

POST /_security/api_key
{
  "name": "my-api-key1",
  "role_descriptors": {
    "my-role": {
      "cluster": ["all"],
      "index": [
        {
          "names": ["cyberdocs"],
          "privileges": ["all"]
        }
      ]
    }
  }
}

In [56]:
from langchain.embeddings import OllamaEmbeddings
from elasticsearch import Elasticsearch
import ollama
# MODEL_NAME = "all-MiniLM-L6-v2"  # small and fast
# MODEL_NAME='nomic-embed-text'
# ES_INDEX = "cvedocs"
# # model = SentenceTransformer(MODEL_NAME)
# # model = OllamaEmbeddings(model=MODEL_NAME)

# def ollama_embedding_function(texts):
#     """
#     texts: List[str] → List[List[float]]
#     """
#     if isinstance(texts, str):
#         texts = [texts]
#     embeddings = []
#     for text in texts:
#         response = ollama.embeddings(model="nomic-embed-text", prompt=text)
#         embeddings.append(response['embedding'])
#     return embeddings[0]

from transformers import AutoTokenizer, AutoModel
import torch

model_name = "markusbayer/CySecBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Mean pooling over last hidden state
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings[0].tolist()

# query_vector = embed_text("What is the CVE related to RDP privilege escalation?")




Some weights of BertModel were not initialized from the model checkpoint at markusbayer/CySecBERT and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import AutoModel
from transformers import AutoTokenizer, AutoModel

import torch
model_name = "ibm-research/CTI-BERT"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def embed_text(text):
    tokens = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        output = model(**tokens)
    # Mean pooling of last hidden state
    return output.last_hidden_state.mean(dim=1).squeeze().tolist()

# vector = get_embedding("CVE-2023-23397 is exploited in phishing campaigns.")
# print(len(vector))  # Should be 768-dim vector

Some weights of BertModel were not initialized from the model checkpoint at ibm-research/CTI-BERT and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
import re
import html
from urllib.parse import unquote


def parse_cpe(cpe_uri):
    """
    Parse a CPE 2.3 URI and return a dictionary of components.
    Example: cpe:2.3:a:vendor:product:version:*:*:*:*:*:*:*
    """
    parts = cpe_uri.split(":")
    if len(parts) >= 7:
        return {
            "type": parts[2],         # a / o / h
            "vendor": unquote(parts[3]),
            "product": unquote(parts[4]),
            "version": unquote(parts[5])
        }
    return {}


def extract_cve_data(json_data, embed_model=None):
    try:
        cve_id = json_data.get("cveMetadata", {}).get("cveId", "")
        containers = json_data.get("containers", {})
        cna = containers.get("cna", {})

        # Description
        description = next(
            (d.get("value") for d in cna.get("descriptions", []) if d.get("lang") == "en"),
            ""
        )
        description = html.unescape(description.strip())

        # CVSS metrics
        metrics = cna.get("metrics", [])
        cvss = {}
        for m in metrics:
            if "cvssV3_1" in m:
                cvss = m["cvssV3_1"]
                break

        # CWE
        cwes = []
        for pt in cna.get("problemTypes", []):
            for desc in pt.get("descriptions", []):
                if desc.get("lang") == "en":
                    cwes.append({
                        "id": desc.get("cweId"),
                        "description": desc.get("description")
                    })

        # Affected packages
        affected_packages = []
        for a in cna.get("affected", []):
            vendor = a.get("vendor", "")
            product = a.get("product", "")
            version_info = a.get("versions", [])
            for v in version_info:
                pkg = {
                    "vendor": vendor,
                    "product": product,
                    "version": v.get("version"),
                    "lessThanOrEqual": v.get("lessThanOrEqual"),
                    "version_type": v.get("versionType"),
                    "status": v.get("status")
                }
                # Optionally include parsed CPE info if available
                for cpe_entry in a.get("cpe", []):
                    cpe_data = parse_cpe(cpe_entry.get("cpe23Uri", ""))
                    pkg.update(cpe_data)
                affected_packages.append(pkg)

        # Infer fix versions
        fix_versions = sorted(
            {f"> {pkg['lessThanOrEqual']}" for pkg in affected_packages if pkg.get("lessThanOrEqual")}
        )
        fix_version = fix_versions[0] if fix_versions else None

        # References
        references = [r.get("url") for r in cna.get("references", []) if "url" in r]

        # Timeline
        timeline = [
            {"date": t["time"], "event": t["value"]}
            for t in cna.get("timeline", []) if "time" in t and "value" in t
        ]

        # Credits
        credits = [c.get("value") for c in cna.get("credits", []) if c.get("lang") == "en"]

        # SSVC (CISA enrichment)
        ssvc_data = {}
        for adp in containers.get("adp", []):
            for metric in adp.get("metrics", []):
                if "other" in metric and metric["other"].get("type") == "ssvc":
                    content = metric["other"].get("content", {})
                    ssvc_data = {
                        "role": content.get("role"),
                        "version": content.get("version"),
                        "exploitStatus": content.get("exploitStatus")
                    }
                    for opt in content.get("options", []):
                        ssvc_data.update(opt)
                    break



        # Final output
        return {
            "id": cve_id,
            "text": cve_id+" "+description,
            "vector": embed_model(cve_id+" "+cna.get("title", "")+" "+description),
            "metadata": {
                "id": cve_id,
                "title": cve_id + " " + cna.get("title", ""),
                "published_date": json_data.get("cveMetadata", {}).get("datePublished", ""),
                "last_updated": json_data.get("cveMetadata", {}).get("dateUpdated", ""),
                "cvss_score": cvss.get("baseScore"),
                "cvss_vector": cvss.get("vectorString"),
                "severity": cvss.get("baseSeverity"),
                "cwe": cwes,
                "affected_packages": affected_packages,
                "fix_version": fix_version,
                "references": references,
                "timeline": timeline,
                "credits": credits,
                "assigner": json_data.get("cveMetadata", {}).get("assignerShortName"),
                "ssvc": ssvc_data
            },
            "sources": references
        }

    except Exception as e:
        print(f"Failed to parse CVE JSON: {e}")
        return {}

In [18]:
import os
from pathlib import Path
import json

def convert_all(input_dir: str, client, batch_size=1000):
    input_path = Path(input_dir)
    all_json_files = list(input_path.rglob("*.json"))
    all_json_files=all_json_files
    print(f"[i] Found {len(all_json_files)} JSON files in '{input_dir}'")

    l1 = []
    total_processed = 0
    total_failed = 0

    for idx, file_path in enumerate(all_json_files):
        try:
            with file_path.open() as f:
                cve_json = json.load(f)
            doc = extract_cve_data(cve_json, embed_text)

            # Skip REJECTED CVEs
            if doc.get("state", "").upper() == "REJECTED":
                continue

            l1.append({"index": {"_index": "cvedocs"}})
            l1.append(doc)
            total_processed += 1

            if len(l1) >= batch_size * 2:
                res = client.bulk(index="cvedocs", operations=l1, refresh=True)
                print(f"[✓] Bulk inserted {batch_size} documents")
                l1 = []  # Clear the batch

        except Exception as e:
            print(f"[!] Failed to process {file_path}: {e}")
            total_failed += 1

    # Final flush if anything remains
    if l1:
        res = client.bulk(index="cvedocs", operations=l1, refresh=True)
        print(f"[✓] Final bulk inserted {len(l1)//2} documents")

    print(f"[✔] Total processed: {total_processed}")
    print(f"[✗] Total failed: {total_failed}")

# Run the batch processor
# convert_all(r"/Users/ar2024/Downloads/cvelistV5-main/cves/2025", client)


## KNN Search

In [20]:
query = "21448"
query_vector = embed_text(query)

response = client.search(index="cvedocs", body={
    "size": 10,
    "knn": {
        "field": "vector",
        "query_vector": query_vector,
        "k": 100,
        "num_candidates": 1000
    }
})



# Display results with score and title
for hit in response["hits"]["hits"]:
    title = hit["_source"]["metadata"].get("title", "No title")
    print(f"{hit['_score']:.2f} - {title}")

0.83 - CVE-2025-21548 
0.83 - CVE-2025-21492 
0.83 - CVE-2025-21491 
0.82 - CVE-2025-21494 
0.82 - CVE-2025-21489 
0.82 - CVE-2025-21498 
0.82 - CVE-2025-21499 
0.82 - CVE-2025-21504 
0.82 - CVE-2025-21544 
0.82 - CVE-2025-21541 


## Vector Ranking,

In [22]:

# Generate query vector
query = " Cve related to CVE-2025-21448 "
query_vector = embed_text(query)  # List[float], e.g., 768-dimensional

# Step 1: Retrieve Top N with ANN (fast)
initial_response = client.search(
    index="cvedocs",
    body={
        "size": 100,
        "knn": {
            "field": "vector",
            "query_vector": query_vector,
            "k": 100,
            "num_candidates": 1000
        }
    }
)

# Get IDs of top-k hits
top_ids = [hit["_id"] for hit in initial_response["hits"]["hits"]]

# Step 2: Rerank with exact cosine similarity
rerank_response = client.search(
    index="cvedocs",
    body={
        "size": 10,
        "query": {
            "script_score": {
                "query": {
                    "ids": {
                        "values": top_ids
                    }
                },
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'vector') + 1.0",
                    "params": {
                        "query_vector": query_vector
                    }
                }
            }
        }
    }
)

# Output ranked results
for hit in rerank_response["hits"]["hits"]:
    score = hit["_score"]
    title = hit["_source"].get("metadata", {}).get("title", "No title")
    print(f"{score:.4f} - {title}")

1.8920 - CVE-2025-21491 
1.8910 - CVE-2025-21498 
1.8893 - CVE-2025-21489 
1.8873 - CVE-2025-21494 
1.8866 - CVE-2025-21492 
1.8797 - CVE-2025-21490 
1.8789 - CVE-2025-21495 
1.8789 - CVE-2025-21499 
1.8741 - CVE-2025-21493 
1.8729 - CVE-2025-21497 


## Multivector Search

In [39]:
from elasticsearch.exceptions import RequestError

query_chunks = [
    "CVE-2025-21955",
    ""
]

# Embed each chunk
query_vectors = [embed_text(chunk) for chunk in query_chunks]

# Multivector average similarity search
try:
    response = client.search(
    index="cvedocs",
    body={
        "size": 10,
        "query": {
            "script_score": {
                "query": {
        "bool": {
          "filter": {
            "exists": { "field": "vector" }
          }
        }
                },
                "script": {
                    "source": """
                        double score1 = cosineSimilarity(params.qv1, 'vector');
                        double score2 = cosineSimilarity(params.qv2, 'vector');
                        return (score1 + score2) / 2 + 1.0;
                    """,
                    "params": {
                        "qv1": query_vectors[0],
                        "qv2": query_vectors[1]
                    }
                }
            }
        }
    }
)
except RequestError as e:
    print("Error:", e.info)

# Display results
for hit in response["hits"]["hits"]:
    print(f"{hit['_score']:.4f} - {hit['_source']['metadata'].get('title', 'No title')}")

1.6522 - CVE-2025-21755 
1.6491 - CVE-2025-21558 
1.6484 - CVE-2025-21555 
1.6484 - CVE-2025-21492 
1.6456 - CVE-2025-21489 
1.6442 - CVE-2025-21582 
1.6428 - CVE-2025-21522 
1.6424 - CVE-2025-21523 
1.6412 - CVE-2025-21757 
1.6407 - CVE-2025-21552 


In [229]:
query = "CVE related to database"
query_vector = embed_text(query) # Assuming model(query) returns a list or array of vectors

# Build the semantic search query
response = client.search(index="cvedocs", body={
    "size": 100,  # Adjust this based on how many results you want
    "query": {
        "script_score": {
            "query": {"match_all": {}},  # Apply cosine similarity on all documents
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'vector') + 1.0",  # +1.0 to avoid negative scores
                "params": {"query_vector": query_vector}
            }
        }
    }
})

# Display the top results with title
for hit in response["hits"]["hits"]:
    title = hit["_source"]["metadata"].get("title", "No title")
    print(f"{hit['_score']:.2f} - {title}")





BadRequestError: BadRequestError(400, 'search_phase_execution_exception', 'runtime error')

## Metadata Search

In [28]:
response = client.search(index="cvedocs", body={
    "size": 2,
    "query": {
        "bool": {
            "filter": [
                {"term": {"metadata.severity": "HIGH"}},
            ]
        }
    }
})

for hit in response["hits"]["hits"]:
    print(f"{hit['_id']} - {hit['_source']['metadata']['severity']} - {hit['_source']['text']}")

XegWZZcBb7rDHA8WNPqf - HIGH - CVE-2025-2631 Out of bounds write vulnerability due to improper bounds checking in NI LabVIEW in InitCPUInformation() that may result in information disclosure or arbitrary code execution.  Successful exploitation requires an attacker to get a user to open a specially crafted VI.  This vulnerability affects NI LabVIEW 2025 Q1 and prior versions.
ZugWZZcBb7rDHA8WNPqf - HIGH - CVE-2025-2735 A vulnerability has been found in PHPGurukul Old Age Home Management System 1.0 and classified as critical. Affected by this vulnerability is an unknown functionality of the file /admin/add-services.php. The manipulation of the argument sertitle leads to sql injection. The attack can be launched remotely. The exploit has been disclosed to the public and may be used.


## Euclidean Distance Search (L2)

In [37]:
query_vector = embed_text("21955")

response = client.search(index="cvedocs", body={
    "size": 100,
    "query": {
        "script_score": {
            "query": {
                "bool": {
                    "filter": {
                        "exists": { "field": "vector" }
                    }
                }
            },
"script": {
  "source": "1 / (1 + l2norm(params.query_vector, 'vector'))",
  "params": {
    "query_vector": query_vector
  }
}
        }
    }
})

for hit in response["hits"]["hits"]:
    title = hit["_source"]["metadata"].get("title", "No title")
    print(f"{hit['_score']:.4f} - {title}")

0.0622 - CVE-2025-21559 
0.0616 - CVE-2025-21555 
0.0615 - CVE-2025-21755 
0.0605 - CVE-2025-21549 
0.0601 - CVE-2025-21553 
0.0599 - CVE-2025-25199 BCryptGenerateSymmetricKey memory leak
0.0597 - CVE-2025-21582 
0.0596 - CVE-2025-21554 
0.0596 - CVE-2025-21558 
0.0595 - CVE-2025-21512 
0.0594 - CVE-2025-21990 drm/amdgpu: NULL-check BO's backing store when determining GFX12 PTE flags
0.0594 - CVE-2025-21545 
0.0594 - CVE-2025-25195 Zulip events can leak private channel names
0.0593 - CVE-2025-21556 
0.0593 - CVE-2025-21502 
0.0593 - CVE-2025-21578 
0.0592 - CVE-2025-21569 
0.0591 - CVE-2025-44894 
0.0591 - CVE-2025-21519 
0.0591 - CVE-2025-21548 
0.0591 - CVE-2025-21544 
0.0591 - CVE-2025-25763 
0.0591 - CVE-2025-3036 yzk2356911358 StudentServlet-JSP Student Management cross site scripting
0.0591 - CVE-2025-21588 
0.0591 - CVE-2025-45240 
0.0590 - CVE-2025-21539 
0.0589 - CVE-2025-21538 
0.0588 - CVE-2025-21546 
0.0588 - CVE-2025-45846 
0.0588 - CVE-2025-21563 
0.0588 - CVE-2025-21579 

In [235]:
query = " what are the exploits will impact disk "
query_vector = model(query)[0]

response = client.search(index="cvedocs", body={
    "size": 100,
    "query": {
        "script_score": {
            "query": {
                "bool": {
                    "filter": [
                        # {"term": {"metadata.severity": "CRITICAL"}},
                        # {"range": {"metadata.cvss_score": {"gte": 9.0}}}
                    ]
                }
            },
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'vector') + 1.0",
                "params": {"query_vector": query_vector}
            }
        }
    }
})

print('Total Document Found : {}'.format(len(response['hits']['hits'])))
for hit in response["hits"]["hits"]:
    print(f"{hit['_id']} - {hit['_source']['metadata']['title']}")

TypeError: string indices must be integers, not 'tuple'

##Euclidean Distance Search (L2)

In [69]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline
import torch
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import requests
from huggingface_hub import configure_http_backend

def backend_factory() -> requests.Session:
    session = requests.Session()
    session.verify = False
    return session

configure_http_backend(backend_factory=backend_factory)

model_name = "markusbayer/CySecBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

# Example: Fill-mask
fill = pipeline("fill-mask", model=model, tokenizer=tokenizer)
print(fill("The latest zero-day CVE-2025-##### exploits a buffer [MASK]"))

text = "The vulnerability allows for remote code execution through buffer overflow in Apache Struts."

# Tokenize input
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)


# Run model inference
with torch.no_grad():
    outputs = model(**inputs)

# Get prediction
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()
print("Predicted class:", predicted_class)

C:\Users\P0107146\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\P0107146\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\P0107146\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\P

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Device set to use cpu


[{'score': 0.5349474549293518, 'token': 2058, 'token_str': 'over', 'sequence': 'the latest zero - day cve - 2025 - # # # # # exploits a buffer over'}, {'score': 0.23651354014873505, 'token': 24672, 'token_str': 'overrun', 'sequence': 'the latest zero - day cve - 2025 - # # # # # exploits a buffer overrun'}, {'score': 0.08803997188806534, 'token': 3091, 'token_str': 'length', 'sequence': 'the latest zero - day cve - 2025 - # # # # # exploits a buffer length'}, {'score': 0.019777385517954826, 'token': 2946, 'token_str': 'size', 'sequence': 'the latest zero - day cve - 2025 - # # # # # exploits a buffer size'}, {'score': 0.007776635233312845, 'token': 1999, 'token_str': 'in', 'sequence': 'the latest zero - day cve - 2025 - # # # # # exploits a buffer in'}]


RuntimeError: a Tensor with 30522 elements cannot be converted to Scalar

In [74]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from langchain.embeddings.base import Embeddings

class CyberBERTEmbeddings(Embeddings):
    def __init__(self, model_name="ibm-research/CTI-BERT"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def embed_documents(self, texts):
        return [self._embed(text) for text in texts]

    def embed_query(self, text):
        return self._embed(text)

    def _embed(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        last_hidden = outputs.last_hidden_state
        mask = inputs['attention_mask'].unsqueeze(-1).expand(last_hidden.shape).float()
        pooled = (last_hidden * mask).sum(1) / mask.sum(1)
        return pooled[0].cpu().numpy().tolist()


## 🔹 Step-by-step: Mean Pooling (Best for Most Cases)

In [11]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Load Cybersecurity-specific BERT model
model_name = "ibm-research/CTI-BERT"  # or "markusbayer/CySecBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Sentence(s) to embed
sentences = [
    "This CVE allows privilege escalation on Windows systems.",
    "The attacker uses DLL injection to execute arbitrary code."
]

# Tokenize


def encode_sentences(model, tokenizer, texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    hidden = outputs.last_hidden_state
    mask = inputs['attention_mask'].unsqueeze(-1).expand(hidden.shape).float()
    return (hidden * mask).sum(1) / mask.sum(1)


vectors = encode_sentences(model, tokenizer, ["APT29 used PowerShell for credential dumping"])




Some weights of BertModel were not initialized from the model checkpoint at ibm-research/CTI-BERT and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##  Full Example: Match Query Sentence to Most Similar One

In [34]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load CTI-BERT model
model_name = "ibm-research/CTI-BERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Encode sentences using mean pooling over the last hidden state
def encode_sentences(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    hidden = outputs.last_hidden_state
    mask = inputs['attention_mask'].unsqueeze(-1).expand(hidden.shape).float()
    return (hidden * mask).sum(1) / mask.sum(1)  # mean pooling

# Reference cybersecurity-related sentences
reference_sentences = [
    "This CVE allows privilege escalation on Windows systems.",
    "The attacker uses DLL injection to execute arbitrary code.",
    "APT28 exploits MS Office vulnerabilities for initial access.",
    "Mimikatz is used to dump credentials from LSASS memory.",
    "Ransomware encrypts files and demands Bitcoin payment.",
    "IBM DB2"
]

# Encode reference sentences
reference_vectors = encode_sentences(reference_sentences)

# Query sentence
query_sentence = "which is database"

# Encode query
query_vector = encode_sentences([query_sentence])

# Compute cosine similarity
similarities = cosine_similarity(query_vector.numpy(), reference_vectors.numpy())[0]

# Find best match
best_match_index = np.argmax(similarities)
print(f"\nQuery: {query_sentence}")
print(f"Best match: {reference_sentences[best_match_index]}")
print(f"Cosine similarity: {similarities[best_match_index]:.4f}")


Some weights of BertModel were not initialized from the model checkpoint at ibm-research/CTI-BERT and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Query: which is database
Best match: IBM DB2
Cosine similarity: 0.5813


In [39]:
query_vector

tensor([[ 2.0588e-01, -1.5686e-01, -4.0028e-01,  6.1085e-03, -1.3478e-01,
         -3.0049e-01,  3.4998e+00,  7.2817e-01,  5.6363e-01,  3.1650e-01,
         -1.1842e-01,  2.4403e-01,  1.6834e-01,  4.0903e-01,  1.5519e-01,
          1.3938e+00, -2.6725e-01, -3.0429e-01, -3.5484e-01,  4.9077e-01,
          9.9984e-02,  8.3340e-02,  7.7137e-01, -7.5173e-01,  7.6474e-01,
         -3.4464e-01, -2.6949e-01,  1.5015e-01, -1.1840e+00,  2.1454e-01,
         -2.2773e-01, -1.7058e-02,  5.1215e-01, -1.0415e+00,  7.0703e-01,
          1.2329e+00,  1.0836e-01,  7.1254e-01, -6.3746e-01, -1.0261e+00,
         -8.2526e-02, -4.0921e-01,  1.3914e+00, -2.0074e-01, -6.6560e-01,
         -6.0683e-01,  7.2662e-01, -7.3028e-01, -3.8009e-01,  2.6628e-01,
          7.1369e-01, -3.0348e-01,  3.1823e-02,  2.5347e-01,  2.3546e-01,
         -3.3531e-01, -4.4245e-01,  4.7088e-01,  2.5162e-01,  7.4056e-01,
          3.1947e-01, -9.0251e-02,  7.5560e-01, -9.5053e-01, -6.7257e-01,
          4.3158e-01, -7.6670e-01, -1.

In [40]:
import torch
import torch.nn.functional as F

# Two example torch vectors
v1 = torch.tensor([0.1, 0.2, 0.3])
v2 = torch.tensor([0.1, 0.25, 0.35])

# Cosine similarity
cos_sim = F.cosine_similarity(v1.unsqueeze(0), v2.unsqueeze(0))
print(cos_sim.item())  # Outputs scalar similarity


0.9986253380775452


# M

In [57]:
import re
def extract_cves(text):
    """
    Extract all CVE identifiers from a string.
    Example formats: CVE-2024-1234, cve-2025-00001, etc.
    """
    cve_pattern = r'CVE-\d{4}-\d+'  # CVE-YYYY-NNNN to NNNNNNN
    return re.findall(cve_pattern, text.upper(), flags=re.IGNORECASE)

text = "Check CVE-2023-123andcve-2024-09876 and CVE-1999-00001?"
print(extract_cves(text))

['CVE-2023-123', 'CVE-2024-09876', 'CVE-1999-00001']


In [71]:
cve_id_list = ["CVE-2024-09876", "CVE-2023-123"]

query = {
    "query": {
        "terms": {
            "metadata.id": cve_id_list
        }
    },
    "_source": [
        "metadata.title",
        "text",
        "metadata.published_date",
        "metadata.severity",
        "metadata.affected_packages"
    ],
}

response = client.search(index="cvedocs", body=query)
response

ObjectApiResponse({'took': 0, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}})